# 文本分类实例

In [1]:
from abc import ABC
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [2]:
# 加载数据集
import pandas as pd
csv_file_path = "ChnSentiCorp_htl_all.csv"
data = pd.read_csv(csv_file_path)
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [3]:
# 数据清洗
data = data.dropna()
data

In [4]:
# 创建dataset
from torch.utils.data import Dataset

class CommentDataset(Dataset):
    def __init__(self, csv_path):
        self.data = pd.read_csv(csv_path).dropna()
        # self.data = self.data.dropna()

    def __getitem__(self, item):
        # 此处的comment仍是文本不是tensor
        # 单条处理比不过batch处理速度，所以传入DataLoader之后再进行处理
        # fixme 此索引方式在进行去重后会出现问题，进行更改
        # label = self.data["label"][item]
        # comment = self.data["review"][item]
        return self.data.iloc[item]["review"], self.data.iloc[item]["label"]

    def __len__(self):
        return len(self.data)

In [5]:
# 测试数据集
dataset = CommentDataset(csv_file_path)
for i in range(5):
    print(dataset[i])
dataset.__len__()

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', 1)
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', 1)


7765

In [6]:
# 进行数据集的划分
from torch.utils.data import random_split
train_set, valid_set = random_split(dataset, lengths=[0.9, 0.1])
len(train_set), len(valid_set)

(6989, 776)

In [7]:
# 加载数据集
from torch.utils.data import DataLoader
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=64, shuffle=False)

In [8]:
# 尝试改写collate_fn使得文本转变为tensor
# 先看传入collate_fn的是什么格式的数据
def test_collate(batch):
    print(batch)
    print(len(batch))

train_loader = DataLoader(train_set, batch_size=32, shuffle=True, collate_fn=test_collate)
for t1, t2 in enumerate(train_loader):
    break

[('非常好！一进房间就能闻到一股花得芳香~~~服务态度十分热情~~~酒店里不但有免费得室内室外温泉泡~~而且还送两张188元得温泉泡~~有不下七十多种温泉：如白酒温泉，绿茶温泉等十分不错~~早饭也相当丰盛可以说应有尽有~~鸡翅特别美味~~通过携程预订还送免费得两杯饮料~~~sogood~~是标准得五星酒店~~~如果觉得离市区远了点可以乘班车或打车（40元左右）~~但绝对物超所值~~走时还送了两串玛瑙手机链~~~下次再去厦门一定要再去~~Imissyou！！', 1), ('房间尚可，环境也不错，只是周边环境不够理想！稍微差了点！', 1), ('我住的158的房,觉得还可以,房间正对酒店大门口的马路,隔音还可以,不吵.卫生间也不是太少,就是设计不太合理,洗澡的龙头力道太大,水打在身上都疼.综合来说还可以.', 1), ('总体感觉不错，性价比挺高的，旁边有个银座，银座下面还有个超市', 1), ('地点在十字路口边上，十分嘈杂，巴士车呼啦啦直到夜里11点。房间很小，洗漱用品不全：沐浴液、洗发液和洗手液是三合一的，很不好用。服务态度不好。电话查询时总是让你听音乐等待很长时间，然后无故将你电话挂断。多算了我一份早餐。', 0), ('地理位置较优越，房间内干净整洁，不能说很安静但出门在外凡事不能太叫真，隔音还是可以的，可以考虑无烟楼层的房间较安静，总台小姐的服务态度还是可以的，外出旅游住宿可以建议考虑此酒店，去王兴记吃虾肉馄饨或去三凤桥买酱排骨还是很方便的，谢谢！', 1), ('如果去成都,在这里居住不错,行程方便,其他的都很差.最气的是请客人吃饭结完帐,很久以后到房间说台布被烫了,硬是要赔500大洋.', 0), ('10月初回黑龙江的时候住了一晚，我所有的评价都是基于齐齐哈尔本地的情况做的，我觉得没办法和北京、上海等大城市的同级别酒店比较。周边环境还不错，楼下基本随时有出租车等着，附近还有商场和超市。服务很好，主动热情，雨伞忘服务台了，很主动的提醒我。没有骚扰电话。稍微有点噪音，走廊的服务台的电话声偶尔能听到。没带电脑，网络情况不知道。因为火车时间的问题，本来打算延迟一小时退房，服务台主动给延长了2个小时。中央空调也还好，不知道冬天会怎么样，淋浴热水很好。', 1), ('我住的是两室一厅的房间,在17楼.虽然只住了一夜,但总体来说,对这个公寓感觉不错.就是有几

In [9]:
# 根据上述输出结果可知，传入collate_fn的数据为一个batch的数据
# 格式为list[tuple(data, label) * batch_size]
# 此部分编写实现将comment通过分词器转为tensor的fn
import torch
tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")

def collate_function(batch):
    comments, labels = [], []
    # batch为list，依次访问进行存储
    for i in batch:
        comments.append(i[0])
        labels.append(i[1])

    inputs = tokenizer(comments, max_length=128, padding="max_length", truncation=True, return_tensors="pt")
    inputs["labels"] = torch.tensor(labels)
    return inputs

In [10]:
train_loader = DataLoader(train_set, batch_size=32, shuffle=True, collate_fn=collate_function)
valid_loader = DataLoader(valid_set, batch_size=64, shuffle=False, collate_fn=collate_function)

next(enumerate(train_loader))[1]

{'input_ids': tensor([[ 101, 4507,  754,  ...,  510,  671,  102],
        [ 101, 3302, 1218,  ...,    0,    0,    0],
        [ 101, 2682, 2454,  ...,    0,    0,    0],
        ...,
        [ 101, 5381,  677,  ..., 3952, 4381,  102],
        [ 101,  122,  510,  ...,    0,    0,    0],
        [ 101, 1762, 3315,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 0, 0, 1])}

In [15]:
from torch.optim import Adam

# 模型及优化器定义
model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3")
optimizer = Adam(model.parameters(), lr=0.00002)

if torch.cuda.is_available():
    model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# 模型训练
def train(epochs, log_step=10):
    for epoch in range(epochs):
        step=0
        model.train()
        print(f"<<<<<<<<Training on epoch{epoch} >>>>>>>>")
        for batch in train_loader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}

            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()

            if step % log_step == 0:
                print(f"global_step: {step}, loss: {output.loss.item()}")
            step += 1
        evaluate()


# 模型验证
def evaluate():
    model.eval()
    acc_num = 0
    # 该方法比no_grad()更加的优化了测试的内存管理方面
    with torch.inference_mode():
        for batch in valid_loader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
                output = model(**batch)
                pred = torch.argmax(output.logits, dim=-1)
                for p, l in zip(pred, batch["labels"]):
                    if p == l:
                        acc_num += 1
        print(f"acc: {acc_num / len(valid_set)}")


In [16]:
train(epochs=5)

<<<<<<<<Training on epoch0 >>>>>>>>
global_step: 0, loss: 0.6822832226753235
global_step: 10, loss: 0.5704638361930847
global_step: 20, loss: 0.48738840222358704
global_step: 30, loss: 0.5126784443855286
global_step: 40, loss: 0.36928489804267883
global_step: 50, loss: 0.4921889305114746
global_step: 60, loss: 0.2984669506549835
global_step: 70, loss: 0.41466575860977173
global_step: 80, loss: 0.2597431242465973
global_step: 90, loss: 0.44846025109291077
global_step: 100, loss: 0.2612557113170624
global_step: 110, loss: 0.2734362781047821
global_step: 120, loss: 0.2482375055551529
global_step: 130, loss: 0.3017747402191162
global_step: 140, loss: 0.2814706563949585
global_step: 150, loss: 0.2245887815952301
global_step: 160, loss: 0.1818622350692749
global_step: 170, loss: 0.1783565878868103
global_step: 180, loss: 0.3004911243915558
global_step: 190, loss: 0.23544727265834808
global_step: 200, loss: 0.11210673302412033
global_step: 210, loss: 0.2140013426542282
acc: 0.9085051546391752

In [18]:
# 预测
test = "这家酒店不错，饭很好吃"
with torch.inference_mode():
    test_tensor = tokenizer(test, return_tensors="pt")
    test_tensor = {k: v.cuda() for k, v, in test_tensor.items()}
    test_result = model(**test_tensor).logits
    pred = torch.argmax(test_result, dim=-1)
    print(pred.item())

1


In [19]:
# 创建pipeline
from transformers import pipeline

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device="cuda:0")

Device set to use cuda:0


In [20]:
pipe("这家酒店不错，饭很好吃")

[{'label': 'LABEL_1', 'score': 0.9989665746688843}]